<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_200DMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-02-23 18:53:18


In [11]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [12]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['TFrame'] == 'D']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [13]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [14]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [15]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      #mid_window = 50
      long_window = 200
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_DMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_DMA'])*100/stock_df['200_DMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):

    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] < 30) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev = df_prec_dev[df_prec_dev['Signal'] != 0]

    return df_prec_dev.tail(1)

def get_daily_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_buy_sell = df_buy_sell.append(df_tmp)


    df_buy_sell['Signal'] = df_buy_sell['Signal'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()

df_daily_buy_sell = get_daily_buy_sell()

In [16]:
df_daily_buy_sell[df_daily_buy_sell['Signal'] == 'Buy'].tail(20)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Criteria
Date,,,,,,,,,,,
2023-08-18 00:00:00+05:30,1751.0,1840.0,-4.85,KOTAKBANK.NS,2207.80,1646.89,6.31,26.10,29.0,Buy,V40
2023-08-31 00:00:00+05:30,241.0,243.0,-0.76,HINDPETRO.NS,575.00,208.30,15.77,138.44,28.0,Buy,VR
2023-10-26 00:00:00+05:30,506.0,545.0,-7.19,DABUR.NS,638.90,500.06,1.16,26.30,19.0,Buy,V40
2023-10-26 00:00:00+05:30,36896.0,38773.0,-4.84,PAGEIND.NS,53197.54,35768.78,3.15,44.18,29.0,Buy,V40
2023-10-26 00:00:00+05:30,909.0,914.0,-0.59,ICICIBANK.NS,1062.70,817.17,11.19,16.95,28.0,Buy,V40
2023-10-26 00:00:00+05:30,2295.0,2470.0,-7.07,PIDILITIND.NS,2889.66,2258.62,1.63,25.89,26.0,Buy,V40
2023-10-26 00:00:00+05:30,2226.0,2305.0,-3.41,RELIANCE.NS,2987.25,2024.60,9.97,34.17,25.0,Buy,V40
2023-11-01 00:00:00+05:30,2343.0,2382.0,-1.66,AKZOINDIA.NS,2879.59,2075.77,12.85,22.93,27.0,Buy,V40
2023-11-01 00:00:00+05:30,1238.0,1274.0,-2.84,HAVELLS.NS,1452.61,1137.49,8.82,17.35,29.0,Buy,V40


In [17]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [18]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)